## 1. Query the HW1 Part2 file using SQL
### a. Install and use the https://pypi.org/project/pandasql/ package

In [ ]:
#!pip install -U pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

### Quick Demo for Relative Path:

* Starting with ``/`` returns to the root directory and starts there

* Starting with ``../`` moves one directory backward and starts there

* Starting with ``../../`` moves two directories backward and starts there (and so on...)

To move forward, just start with the first sub directory and keep moving forward.

In [1]:
import os

# get current file location
path = os.getcwd()
path = os.path.abspath(os.path.dirname('..'))
path

'/content'

In [ ]:
# go back the upper level
path = os.path.abspath(os.path.dirname(os.path.dirname('__file__')))
path = os.path.abspath(os.path.dirname(os.getcwd()))
path = os.path.abspath(os.path.join(os.getcwd(), ".."))
path

'C:\\Users\\xxxli\\Desktop\\AltDataGA.2049.zl2819\\AltDataGA.2049.zl2819\\src'

In [ ]:
# go two level backward 
path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
path

'C:\\Users\\xxxli\\Desktop\\AltDataGA.2049.zl2819\\AltDataGA.2049.zl2819'

In [ ]:
RELATIVE_PATH_1 = os.path.join(path,'data','alt_data_hw1_prt1_enrgz.csv')
RELATIVE_PATH_2 = os.path.join(path,'data','alt_data_hw1_prt2_all_trans_500k.csv')
RELATIVE_PATH_1

'C:\\Users\\xxxli\\Desktop\\AltDataGA.2049.zl2819\\AltDataGA.2049.zl2819\\data\\alt_data_hw1_prt1_enrgz.csv'

##### Import Data:

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# mypath = r'~\Desktop\Alternative Data\HW\HW1\data\alt_data_hw1_prt2_all_trans_500k.csv'
# myrealtivepath = r'..\data\alt_data_hw1_prt2_all_trans_500k.csv'
p2_df = pd.read_csv(RELATIVE_PATH_2).dropna(0)
p2_df

,DATE_KEY,STORE,PRICE_PAID,QTY,DESCRIPTION,DEPARTMENT,CATEGORY
0,20170214.0,ff869707c585427f5a1150e6acdbb2edfff045d3,1.67,1.0,HERB-OX CHKN BOUIL GRAN,GROCERY,SOUP DRY/BOUILLON
1,20170214.0,231ffad690bd3aa5f2c81f303e9c2adda57dba2a,2.50,1.0,NAB BELVITA BLUEBERRY,GROCERY,COOKIES
2,20170214.0,338c90a92703191138bd6e44224f3335ff7ef6a0,3.00,1.0,RAISN BRAN CRUNCH,GROCERY,CEREAL RTE
3,20170214.0,e87d15f4c44f276f7834f14d944386082dd46238,1.00,2.0,CAMP HOME STYLE CHIX ND,GROCERY,SOUP CONDENSED
4,20170214.0,85e200307626f18773d40795f4e2c266003a16b4,2.00,4.0,CRYS LT GRN TEA PCH MNG,GROCERY,DRINK MIX/POWDERED
...,...,...,...,...,...,...,...
499994,20170214.0,38642bdb5c115dd3ee4d73fe74e0ff273f1a3a10,1.89,1.0,DASANI 20OZ SINGLE,GROCERY,WATER DRINKING
499995,20170214.0,79a44bf7e5c526cb1ebe00a740eb9cdc3c2ffe8b,15.49,1.0,MICH ULTRA 18 PK LN,ALCOHOL,BEER
499996,20170214.0,1bfe822b6c4d42c0b8cde70327cbd57ea1427fee,3.29,1.0,HIGH LIFE 12PK CAN,ALCOHOL,BEER
499997,20170214.0,5d6b7b902c7eaa05540e6e0dacc01c93c1bdc06d,0.50,1.0,SMARTFOOD,GROCERY,SNACKS CHIPS


### b. Create SQL select statements which achieve the following:
##### i. Count of unique descriptions as well as count of transactions per unique description as a % of total number of transactions.
        1. Columns:
            a. Description,
            b. count unique descriptions,
            c. count as % of total transactions (to calculate total 
               transactions, use the quantity column) – try to use a window 
               function here, but if it doesn’t work, try another solution
    
        2. Rows: For each unique description, count the number of times it 
        occurs in the 500k Part2 file

In [ ]:
q_a = '''
SELECT Description as Description
FROM p2_df
'''
pysqldf(q_a).head()

,Description
0,HERB-OX CHKN BOUIL GRAN
1,NAB BELVITA BLUEBERRY
2,RAISN BRAN CRUNCH
3,CAMP HOME STYLE CHIX ND
4,CRYS LT GRN TEA PCH MNG


In [ ]:
q_b = '''
SELECT COUNT( DISTINCT Description ) as CountOfUniqueDescriptions
FROM p2_df
'''
pysqldf(q_b)

,CountOfUniqueDescriptions
0,8569


In [ ]:
q_c = '''
SELECT Description, 
    (SELECT SUM(Qty) FROM p2_df) as total, 
    (SUM(Qty)) as Quantity, 
    ((SUM(Qty))/(SELECT SUM(Qty) FROM p2_df)) * 100 as Percentage
FROM p2_df
GROUP BY Description
'''
pysqldf(q_c).head()

,DESCRIPTION,total,Quantity,Percentage
0,(D)BECKS 24PK BTL,665959.0,10.0,0.001502
1,(D)BEST DAMN ROOT 6PK BTL,665959.0,11.0,0.001652
2,(D)BEST DAMN SODA 6PK BTL,665959.0,6.0,0.000901
3,(D)HENRYS CHERRY 6PK BTL,665959.0,29.0,0.004355
4,(D)HENRYS GINGER 6PK BTL,665959.0,14.0,0.002102


In [ ]:
q_i = '''
SELECT Description as Description, 
        ((SUM(Qty))/(SELECT SUM(Qty) FROM p2_df)) * 100 as Percentage,
        COUNT (Description) as occurances
FROM p2_df
GROUP BY Description;
'''
pysqldf(q_i).head()

,Description,Percentage,occurances
0,(D)BECKS 24PK BTL,0.001502,8
1,(D)BEST DAMN ROOT 6PK BTL,0.001652,10
2,(D)BEST DAMN SODA 6PK BTL,0.000901,6
3,(D)HENRYS CHERRY 6PK BTL,0.004355,23
4,(D)HENRYS GINGER 6PK BTL,0.002102,14


##### ii. Select the top 10 product descriptions by sales volume (price*quantity) per store
        
        1. Columns
            a. Store (StoreID)
            b. Description
            c. Total Sales Volume
        
        2. Rows: For each unique store ID, there should be 10 rows of 
        descriptions or the max number of unique descriptions that the store 
        has in the file if the max is less than 10

In [ ]:
s_1 = '''
SELECT COUNT( DISTINCT Store) as Store
FROM p2_df
'''
pysqldf(s_1).head(20)

,Store
0,982


In [ ]:
s_2 = '''
SELECT Store as Store, 
        Description as Description,
        SUM(QTY) as Quantity,
        Price_Paid as SinglePrice,
        SUM(QTY) * Price_Paid as SalesVolume
FROM p2_df
GROUP BY Store, Description
ORDER BY Store, SalesVolume DESC
'''
pysqldf(s_2).head(20)

,Store,Description,Quantity,SinglePrice,SalesVolume
0,105d2d9dd58f37a447d362d466c9d8cae32a88ba,SMIRNOFF 80PR,3.0,17.99,53.97
1,105d2d9dd58f37a447d362d466c9d8cae32a88ba,SEAGRAM CROWN ROYAL,1.0,39.99,39.99
2,105d2d9dd58f37a447d362d466c9d8cae32a88ba,TANQUERAY GIN94.6 PRF,1.0,30.99,30.99
3,105d2d9dd58f37a447d362d466c9d8cae32a88ba,RUMPLEMINZ PEP SCHNP100,1.0,25.99,25.99
4,105d2d9dd58f37a447d362d466c9d8cae32a88ba,NATURAL LIGHT 18PK,2.0,12.99,25.98
5,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CROWN ROYAL REGAL APPLE,1.0,20.99,20.99
6,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CROWN ROYAL,1.0,20.99,20.99
7,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CROWN HARVEST RYE,1.0,20.99,20.99
8,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CAPTMORGAN RUM,1.0,20.99,20.99
9,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CAPTAIN MORGAN WHITE,1.0,20.99,20.99


In [ ]:
q_ii = '''

SELECT *
FROM (

SELECT *, RANK() OVER (
    PARTITION BY StoreID
    ORDER BY Total_Sales_Volume DESC
) as rank

FROM(
    SELECT Store as StoreID, 
            Description as Description,
            SUM(QTY) * Price_Paid as Total_Sales_Volume
    FROM p2_df
    GROUP BY Description, StoreID
    ORDER BY StoreID
)

)
WHERE rank <=10
'''
pysqldf(q_ii).head(30)

,StoreID,Description,Total_Sales_Volume,rank
0,105d2d9dd58f37a447d362d466c9d8cae32a88ba,SMIRNOFF 80PR,53.97,1
1,105d2d9dd58f37a447d362d466c9d8cae32a88ba,SEAGRAM CROWN ROYAL,39.99,2
2,105d2d9dd58f37a447d362d466c9d8cae32a88ba,TANQUERAY GIN94.6 PRF,30.99,3
3,105d2d9dd58f37a447d362d466c9d8cae32a88ba,RUMPLEMINZ PEP SCHNP100,25.99,4
4,105d2d9dd58f37a447d362d466c9d8cae32a88ba,NATURAL LIGHT 18PK,25.98,5
5,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CAPTAIN MORGAN WHITE,20.99,6
6,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CAPTMORGAN RUM,20.99,6
7,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CROWN HARVEST RYE,20.99,6
8,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CROWN ROYAL,20.99,6
9,105d2d9dd58f37a447d362d466c9d8cae32a88ba,CROWN ROYAL REGAL APPLE,20.99,6


----------------
## 2. Regular Expressions
### a. Use both the HW1Part1 and HW1Part2 files. Use the RE module
Import Data:

In [ ]:
# part1 = r'~\Desktop\Alternative Data\HW\HW1\data\alt_data_hw1_prt1_enrgz.csv'
# part2 = r'~\Desktop\Alternative Data\HW\HW1\data\alt_data_hw1_prt2_all_trans_500k.csv'
df_enrgz = pd.read_csv(RELATIVE_PATH_1).dropna(0)
df_all = pd.read_csv(RELATIVE_PATH_2).dropna(0)
#df_enrgz

### b. Devise 1 regular expression which matches captures the products which look like Energizer (30 / 33 unique descriptions in the Part1 file look like Energizer transactions.

##### Operating in LIST:

In [ ]:
mylist = df_enrgz.DESCRIPTION.unique().tolist()
mystring = '\n'.join(mylist)
# print(mylist)
# print('-'*20)
# print(mystring)
print('There are ',len(mylist),' unique drescriptions in the file.')

There are  33  unique drescriptions in the file.


In [ ]:
import re
mypattern = '(EN.\S.+)'
y = re.findall(mypattern ,mystring)
print("The number of unique Energizer product I found = ",len(y),"/",len(mylist))
print('----- Find all results: -----')
for item in y:
    print(item.replace('\n',''))

The number of unique Energizer product I found =  30 / 33
----- Find all results: -----
ENRGZ MAX AAA
ENRGZ BATTERY AAA
ENRGZ MAX AA
ENRGZ PHO LITH 3V 123A
ENER ECO ADVNC AA 4CT BATTERY
ENRGZ H/AID AZ10DP-8
ENERGIZER ULT LITH 9V
ENRGZ BATTERY AA
ENRGZ H/AID AZ13DP-8
ENRGZ H/AID AZ312DP-8
ENRGZ MAX AA E91MP-8
ENRGZ MAX AAA E92MP-8
ENER ALK E90 2PK
ENGZR N E90-1.5V
ENRGZ MAX 9V 522BP-2
ENRGZ 3V WTCH LITH 2025
ENRGZ RECHARGE 4CT AAA
ENGZR ZINC AIR AZ312DP
ENRGZ MAX D E95BP-4
ENRGZ 3V WTCH LITH 2016
ENGZR VALUE CHARGER
ENRGZ 8X ULT LITHIUM-AA
ENRGZ 6V LANTERN BAT
ENRGZ 3V CALC BAT 2450
ENGZR ZINC AIR AZ13DP
ENRGZ WATCH 392 NO MERC
ENRGZ RECHARGE 4CT AA
ENRGZ 3V WTCH LITH 2032
ENRGZ 4X ADV LITHIUM AA
ENRGZ 4X ADV LITHIUM-AA


##### Operating in pd.DataFrame:

In [ ]:
# df unique description only
df2 = df_enrgz.DESCRIPTION[df_enrgz.DESCRIPTION.unique()]
df_2 = pd.DataFrame(df2).reset_index().rename(columns={'index':'Description'}).drop(columns='DESCRIPTION')
df_2

,Description
0,ENRGZ MAX AAA
1,ENRGZ BATTERY AAA
2,ENRGZ MAX AA
3,ENRGZ PHO LITH 3V 123A
4,ENER ECO ADVNC AA 4CT BATTERY
5,ENRGZ H/AID AZ10DP-8
6,ENERGIZER ULT LITH 9V
7,ENRGZ BATTERY AA
8,ENRGZ H/AID AZ13DP-8
9,ENRGZ H/AID AZ312DP-8


In [ ]:
# result:
mypattern2 = '(^EN[EGR]+[0-9a-zA-Z])'
energizer_indices = []
for idx, row in df_2.iterrows():
    if re.search(mypattern2, row['Description']):
        energizer_indices.append(idx)
df_result = df_2.iloc[energizer_indices]
print(len(df_result.index),'/',len(df_2.index),' unique descriptions in the Part1 file look like Energizer transactions.')
df_result

30 / 33  unique descriptions in the Part1 file look like Energizer transactions.


,Description
1,ENER ALK E90 2PK
2,ENER ECO ADVNC AA 4CT BATTERY
3,ENERGIZER ULT LITH 9V
4,ENGZR N E90-1.5V
5,ENGZR VALUE CHARGER
6,ENGZR ZINC AIR AZ13DP
7,ENGZR ZINC AIR AZ312DP
8,ENRGZ 3V CALC BAT 2450
9,ENRGZ 3V WTCH LITH 2016
10,ENRGZ 3V WTCH LITH 2025


### c. Try the regular expression on the large 500k Part2 file. Make sure there are no false positives, i.e. only the real Energizer transactions (from Part1 file) are captured, but not any other ones.

##### Operating in LIST:

In [ ]:
mylist2 = df_all.DESCRIPTION.unique().tolist()
mystring2 = '\n'.join(mylist2)

mypattern3 = '(\nEN[ERG].+)'

y2 = re.findall(mypattern3, mystring2)
print("The number of unique Energizer product I found = ",len(y2),"/",len(mylist2))
print('---- List all results: ----')
for it in y2:
    print(it.replace('\n',''))

The number of unique Energizer product I found =  22 / 8569
---- List all results: ----
ENRGZ BATTERY AAA
ENRGZ MAX AAA E92MP-8
ENRGZ MAX AAA
ENRGZ BATTERY AA
ENRGZ PHO LITH 3V 123A
ENRGZ MAX AA E91MP-8
ENRGZ MAX AA
ENRGZ H/AID AZ312DP-8
ENER ECO ADVNC AA 4CT BATTERY
ENER ALK E90 2PK
ENERGIZER ULT LITH 9V
ENRGZ 3V WTCH LITH 2025
ENRGZ 3V CALC BAT 2450
ENRGZ 3V WTCH LITH 2016
ENRGZ WATCH 392 NO MERC
ENRGZ H/AID AZ10DP-8
ENGZR ZINC AIR AZ13DP
ENRGZ MAX 9V 522BP-2
ENRGZ H/AID AZ13DP-8
ENRGZ 8X ULT LITHIUM-AA
ENRGZ 6V LANTERN BAT
ENRGZ 4X ADV LITHIUM AA


##### Operating in pd.DataFrame:

In [ ]:
df3 = df_all.DESCRIPTION[df_all.DESCRIPTION.unique()]
df4 = pd.DataFrame(df3).reset_index().rename(columns={'index':'Description'}).drop(columns='DESCRIPTION')
df4

,Description
0,HERB-OX CHKN BOUIL GRAN
1,NAB BELVITA BLUEBERRY
2,RAISN BRAN CRUNCH
3,CAMP HOME STYLE CHIX ND
4,CRYS LT GRN TEA PCH MNG
...,...
8564,MOM CHOC MRSHMLW MATE 2X
8565,MHI S/F F/F DCF F/VAN
8566,MGD 6PK NR
8567,MILBEST LT 6PK 16OZ


In [ ]:
energizer_indices_4 = []
mypattern2 = '(^EN[EGR]+[0-9a-zA-Z])'
mypattern3 = '(^EN[ERG].+)'

for idx, row in df4.iterrows():
    if re.search(mypattern2, row['Description']):
        energizer_indices_4.append(idx)
df5 = df4.iloc[energizer_indices_4]
print(len(df5.index),'/',len(df4.index),' unique descriptions in the Part1 file look like Energizer transactions.')
df5.reset_index()

22 / 8569  unique descriptions in the Part1 file look like Energizer transactions.


,index,Description
0,1782,ENRGZ BATTERY AAA
1,2544,ENRGZ MAX AAA E92MP-8
2,3480,ENRGZ MAX AAA
3,3629,ENRGZ BATTERY AA
4,4165,ENRGZ PHO LITH 3V 123A
5,4231,ENRGZ MAX AA E91MP-8
6,4343,ENRGZ MAX AA
7,4844,ENRGZ H/AID AZ312DP-8
8,4985,ENER ECO ADVNC AA 4CT BATTERY
9,5037,ENER ALK E90 2PK


## 3. Text mining
### a. Input is the 500k HW1Part2 file
### b. Use sklearn
### c. Create a bigram (1 and 2 gram) vector space using no stemming, no stop words and a word tokenizer with default settings, strip non alpha numeric and convert to all lowercase.
### d. Create a vector space matrix from the above using countvectorizer. Steps c and d could possibly be done in 1 line of code.

In [ ]:
import pandas as pd
import numpy as np
import string, re
import warnings
warnings.filterwarnings("ignore")
#path3 = r'~\Desktop\Alternative Data\HW\HW1\data\alt_data_hw1_prt2_all_trans_500k.csv'
data = pd.read_csv(RELATIVE_PATH_2, usecols={'DESCRIPTION','CATEGORY'}).dropna() # drop null description cells

corpus = data.DESCRIPTION.unique().tolist()

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (1,2),
                             analyzer = 'word', 
                             # max_features = 50, 
                             # min_df = 5, 
                             lowercase = True)

counts = vectorizer.fit_transform(corpus).toarray()
features = vectorizer.get_feature_names() 
# idx = [(str(i+1)+'th Desp') for i in range(len(corpus))]
df = pd.DataFrame(counts, columns = features, 
                  index = corpus)
# df

### e. Remove ngrams which occur in less than 5 unique descriptions

In [ ]:
df_count = pd.DataFrame(df.sum(0), columns={'Counts'})
delete_features = df_count[df_count['Counts'] < 5].index.tolist()
df_feature_reduced = df.drop(delete_features, axis = 1,inplace=False)
desps = df_feature_reduced.index.values
df_feature_reduced

,10,100,100 nat,1000,10c,10c frtsnk,10ct,10oz,10pk,12,...,yoplait,yoplait grk,zatrn,zero,zero 12pk,zero 4pk,zero 6pk,zesty,zico,zyrtec
HERB-OX CHKN BOUIL GRAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NAB BELVITA BLUEBERRY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RAISN BRAN CRUNCH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CAMP HOME STYLE CHIX ND,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CRYS LT GRN TEA PCH MNG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MOM CHOC MRSHMLW MATE 2X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MHI S/F F/F DCF F/VAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MGD 6PK NR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MILBEST LT 6PK 16OZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### f. Using the counts scores find the most likely product categories, i.e. a collection of descriptions which all represent a single product category. I.e. “bud light” is a popular bigram and all of the descriptions containing those words would be in your answer set.

Present your answer with 3 columns:
    
    i. Ngram (repeats for each description)
    ii. Sum of counts for the ngram
    iii. Description


In [ ]:
df2 = df_feature_reduced.T
df2['Counts'] = df2.sum(1)
count_df = df2[['Counts']]
count_df

,Counts
10,17
100,63
100 nat,9
1000,5
10c,10
...,...
zero 4pk,6
zero 6pk,6
zesty,7
zico,5


-----------
#### Extra Credit: 
##### Since unigram words are always in the bigram, we want to substract the number of bigram count from unigram count:

For example:
* Count of ``'light'`` is $74$


* Count of ``'bud light'`` is $26$
* Count of ``'coors light'`` is $12$
* Count of ``'light 18pk'`` is $7$
* Count of ``'light lime'`` is $5$
* Count of ``'natural light'`` is $7$


* the true counts of ``'light'`` that we want to keep is $74-26-12-7-5-7=17$

In [ ]:
onewordngrams = []
for item in count_df.index.values:
    if ' ' not in item:
        onewordngrams.append(item)
# onewordngrams
twowordngrams = []
for item in count_df.index.values:
    if ' ' in item:
        twowordngrams.append(item)

oneword = 'light'
onewordlist = []
for item in twowordngrams:
    if oneword in item.split(' '):
        onewordlist.append(item)
print(count_df.loc[onewordlist])
sub_count = count_df.loc[onewordlist].sum().item()
print(' ')
print(count_df.loc[oneword].item())
real_count = count_df.loc[oneword].item()-sub_count
real_count

               Counts
bud light          26
coors light        12
light 18pk          7
light lime          5
natural light       7
 
74


17

In [ ]:
def real_count(oneword):
    onewordlist = []
    for item in twowordngrams:
        if oneword in item.split(' '):
            onewordlist.append(item)
    # print(onewordlist)
    sub_count = count_df.loc[onewordlist].sum().item()
    # print(count_df.loc[oneword].item())
    real_count = count_df.loc[oneword].item()-sub_count
    onewordlist.append(oneword)
    return real_count, count_df.loc[onewordlist]

In [ ]:
update_dict = {}
store_list = {}
for oneword in onewordngrams:
    update_dict[oneword] = real_count(oneword)[0]
    store_list[oneword] = real_count(oneword)[1]
# update_dict

--------
##### For each ngram, find out the indices of all the descriptions that contain this ngram:

In [ ]:
def extract_idx(x):
    result = x.nonzero()
    return list(list(result)[0])
idx_df = pd.DataFrame(df_feature_reduced.apply(extract_idx))
idx_df

,0
10,"[163, 2163, 3201, 3861, 4367, 4536, 5127, 5278..."
100,"[226, 269, 494, 594, 764, 1183, 1307, 1429, 14..."
100 nat,"[1473, 3353, 3626, 4405, 4519, 4882, 5249, 614..."
1000,"[255, 1359, 2425, 3533, 4212]"
10c,"[225, 1371, 1495, 1640, 2269, 2560, 2632, 2640..."
...,...
zero 4pk,"[875, 1271, 2357, 5097, 5897, 8357]"
zero 6pk,"[64, 373, 1057, 1209, 3743, 4786]"
zesty,"[38, 1240, 1451, 2538, 2975, 4181, 4758]"
zico,"[379, 1257, 2950, 6250, 6349]"


##### Once obtained indices, we can get the list of DESCRIPTIONs for each ngram:

In [ ]:
def extaxt_desp(x):
    return desps[x[0]]
desps_df = pd.DataFrame(idx_df.apply(extaxt_desp, axis=1), columns={'Descriptions'})
desps_df

,Descriptions
10,"[A-1 SAUCE 10 OZ., LACHOY SOY SAUCE 10 OZ., LI..."
100,"[PFRM 100% WHWHE HRTY BR, YOPLAIT GRK 100 LEMO..."
100 nat,"[CAMP 100%NAT CKN NOODL, PFRM WG 100%NAT GER D..."
1000,"[KRAFT DRSNG 1000 ISLAND, KRAFT FAT FREE 1000 ..."
10c,"[BC FRT SHAPE 10C SCOOBY, KLG 10C FRTSNK MX BE..."
...,...
zero 4pk,"[DAN TRIP ZERO 4PK BERRY, DAN TRIP ZERO 4PK PE..."
zero 6pk,"[COKE ZERO 6PK 500ML, PEPSI ZERO 6PK 500ML, CO..."
zesty,"[MCCRMK SPAGHETTI ZESTY, KRAFT ZESTY ITALIA 16..."
zico,"[ZICO CHOCOLATE 16.9 OZ, ZICO WTRMLN RSP 16.9 ..."


##### Made up a DataFrame:
​
*** For a more clear looking, Multi-index is used here: ***

In [ ]:
ngrams = desps_df.index.tolist()
deps = desps_df['Descriptions'].values.tolist()
array_1 = []
for zword, dep_l in zip(ngrams, deps):
    for i in range(len(dep_l)):
        array_1.append(zword)

deps2 = [list(l) for l in deps]
array_2 = np.concatenate(deps2).tolist()

dic = df2['Counts'].to_dict()
array_3 = []
for a1 in array_1:
    array_3.append(dic[a1])
    
arrays = [array_1, array_3, array_2]
tuples = list(zip(*arrays))
mindex = pd.MultiIndex.from_tuples(tuples, names=['ngram', 'counts','description'])
s = pd.DataFrame([0]*len(tuples),columns={'ignore'}, index=mindex)
answer_before_sort = s.drop(columns='ignore')

In [ ]:
# print out some for a view:
answer_before_sort.tail(50)

Empty DataFrame
Columns: []
Index: [(zero, 82, PWRADE ZERO BERRY 8PK), (zero, 82, COKE ZERO C/F 2LT), (zero, 82, PWRADE ZERO FRPNCH 8P), (zero, 82, MONSTER ABSLTLY ZERO 10), (zero, 82, PWRADE ZERO GRAPE 8PK), (zero, 82, MONSTER ZERO 4P 16Z), (zero, 82, AMP ZERO BLUBRY WH GRP), (zero, 82, COKE CHERRY ZERO 20Z), (zero, 82, SPRITE CHERRY ZERO 20Z), (zero, 82, AMP ZERO WATERMELON), (zero, 82, DAN TRIP ZERO 4PK COCON), (zero, 82, FANTA ZERO ORANGE DRINK), (zero 12pk, 5, COKE ZERO 12PK), (zero 12pk, 5, COKE ZERO C/F 12PK), (zero 12pk, 5, SPRITE DT ZERO 12PK), (zero 12pk, 5, MELLO YELLO ZERO 12PK), (zero 12pk, 5, PIBB ZERO 12PK), (zero 4pk, 6, DAN TRIP ZERO 4PK BERRY), (zero 4pk, 6, DAN TRIP ZERO 4PK PEACH), (zero 4pk, 6, DAN TRIP ZERO 4PK VANLA), (zero 4pk, 6, DAN TRIP ZERO 4PK STRAW), (zero 4pk, 6, DAN TRIP ZERO 4PK BANAN), (zero 4pk, 6, DAN TRIP ZERO 4PK COCON), (zero 6pk, 6, COKE ZERO 6PK 500ML), (zero 6pk, 6, PEPSI ZERO 6PK 500ML), (zero 6pk, 6, COKE ZERO 6PK 7.5OZ), (zero 6pk, 6, SPRITE ZERO 6PK 500ML), (zero 6pk, 6, SPRITE ZERO 6PK 7.5OZ), (zero 6pk, 6, COKE ZERO 6PK 8OZ NR), (zesty, 7, MCCRMK SPAGHETTI ZESTY), (zesty, 7, KRAFT ZESTY ITALIA 16OZ), (zesty, 7, HC ZESTY GUMBO SOUP), (zesty, 7, MCC GMM ZESTY HERB), (zesty, 7, HNZ ZESTY COCKTL SCE), (zesty, 7, ORE IDA ZESTY TWIRLS), (zesty, 7, KRAFT FF ZESTY ITALIAN), (zico, 5, ZICO CHOCOLATE 16.9 OZ), (zico, 5, ZICO WTRMLN RSP 16.9 OZ), (zico, 5, ZICO NATURAL 16.9 OZ), (zico, 5, ZICO NATURAL 1 LTR), (zico, 5, ZICO PINEAPPLE 16.9 OZ), (zyrtec, 9, PSE ZYRTEC D 12HR 24 CT), (zyrtec, 9, ZYRTEC 10 MG TABS), (zyrtec, 9, ZYRTEC CH SF DF ALR GRP), (zyrtec, 9, ZYRTEC CH SF DF ALR BGM), (zyrtec, 9, ZYRTEC 10 MG LIQ GELS), (zyrtec, 9, ZYRTEC DISV TABS CHILDS), (zyrtec, 9, ZYRTEC 70CT ALLERGY TAB), (zyrtec, 9, ZYRTEC ALLERGY LIQCAPS), (zyrtec, 9, ZYRTEC ADLT DISV TABS)]

In [ ]:
len(answer_before_sort.index.get_level_values('description').values)

30787

### g. Limit the above results to a maximum of 50 ngrams

In [ ]:
# sort by counts, and then by ngram just incase the counts are the same:
df_after_sort = answer_before_sort.sort_index(by=['counts','ngram'], ascending=False)
top50_ngrams = df_after_sort.index.get_level_values('ngram').unique()[:50]
top50_ngrams

Index(['choc', 'nab', 'bc', '6pk', 'mcc', '12pk', 'lt', 'pk', 'camp', 'pb',
       'kft', 'dan', 'crm', 'om', 'beef', 'bar', 'wht', 'chicken', 'klg',
       'qkr', 'gtrade', 'orig', '4pk', 'cherry', 'tea', 'mix', 'mini', 'jello',
       'zero', 'can', 'lemon', 'bud', '8pk', 'chkn', 'rice', 'vanilla',
       'light', 'nr', 'chs', 'taco', 'prog', 'sf', 'org', 'cheese', 'si',
       'nat', 'coke', 'white', 'crest', 'mx'],
      dtype='object', name='ngram')

In [ ]:
# extract dataframe by top50 counted ngrams:
answer_top50 = df_after_sort.loc[top50_ngrams.values.tolist()]
answer_top50.to_csv('Q3_Answer.csv')
# print some for a view:
answer_top50.tail(200)

Empty DataFrame
Columns: []
Index: [(white, 67, SCOTTISSUE WHITE 4 CT), (white, 67, SANTITAS CHIP WHITE), (white, 67, PAR SHELLS & WHITE CHED), (white, 67, OM O/R WHITE TURKEY), (white, 67, KLNX FLATBX WHITE), (white, 67, TRIDENT WHITE SPRMNT), (white, 67, VIVA BIG ROLL WHITE), (white, 67, OM HNY SMKD WHITE TURK), (white, 67, BLUE MOON BELGIAN WHITE), (white, 67, SMARTFOOD WHITE CHEDDAR), (white, 67, LIBBY WHLE WHITE POTATO), (white, 67, BC FROST CREAMY WHITE), (white, 67, DOVE 6BAR WHITE), (white, 67, CREST BKS & PEROX WHITE), (white, 67, SM WHITE WINE CHKN & CS), (white, 67, OM SMKD WHITE TURKEY), (white, 67, CHEEZIT WHITE CHEDR CUP), (white, 67, KFT DELX WHITE CHED&BCN), (white, 67, PB FROST RTS WHITE), (white, 67, BC FROST RTS WHP WHITE), (white, 67, PB CAKE WHITE), (white, 67, O LONDON TOAST WHITE), (white, 67, SCOTT BATH TISSUE WHITE), (white, 67, BC COOKIE ICING WHITE), (white, 67, DKB ORGANIC WHITE BREAD D), (white, 67, TOSTITO WHITE CORN), (white, 67, LIBBY SLCD WHITE POTATO), (white, 67, BC CAKE MX WHITE), (white, 67, BAKRS CHOC SQ WHITE), (white, 67, COLG T/P TTL ADV WHITE), (white, 67, KRFT MAC&CHS WHITE CHDR), (white, 67, 15950 CJD LITE WHITE), (white, 67, BC CUPCK ICING WHITE), (white, 67, VIVA WHITE CAS TOWEL 6RL), (white, 67, COLGATE ENML WHITE CLN MNT), (white, 67, SI WHITE PEPPER GROUND), (white, 67, KLNX FLATBX WHITE ULTRA), (white, 67, OEP TACO SHL WHITE CORN), (white, 67, HEINZ WHITE VINEGAR 16Z), (white, 67, CHEEZ-ITS WHITE CHEDDAR), (white, 67, DOVE WHITE BAR SOAP T/S), (white, 67, CAMP CHK W/WHITE W/RICE), (white, 67, SANTITAS STRIPS WHITE), (white, 67, COLG TOTAL PLS WHITE), (white, 67, DOVE 2BAR BATH WHITE), (white, 67, QUAKER WHITE CORNMEAL), (white, 67, BETTY CROCKER WHITE GEL), (white, 67, BC WHITE TUBE ICING), (white, 67, CHEEZ-IT WHITE CHEDDAR), (white, 67, MCC GRMT BLGN WHITE ALE), (white, 67, MT DEW WHITE OUT 20OZ), (white, 67, CBC WHITE BAGELS), (white, 67, SAFEGRD 4BAR WHITE), (white, 67, COLGATE OPTIC WHITE MW), (white, 67, CAPTAIN MORGAN WHITE), (white, 67, HONEST TEA MANGO WHITE), (white, 67, DOVE ADV APPLE & WHITE TEA), (white, 67, WHITE HORSE), (white, 67, G&B CHOC BAR ORG WHITE), (white, 67, BP SMKD WHITE TRKY FRK), (white, 67, QKR GRITS REG WHITE), (white, 67, MT DEW WHITE OUT 12PK), (white, 67, CREST WHITE EXPRSN CINN), (white, 67, JD EGG WHITE FLATBREAD), (white, 67, TOM'S WHITE CLN MINT PASTE), (white, 67, TYSON CHUNK WHITE CHICK), (white, 67, HOME PRIDE WHITE LOAF BRE), (crest, 67, CREST 3DW FOAM CLEAN), (crest, 67, CREST VIVID WHT 3D MNT), (crest, 67, CREST 3D WHT RAD MINT), (crest, 67, CREST T/P W/BAKING SODA), (crest, 67, CREST +SCOPE TUBE), (crest, 67, CREST T/P ANTICAVITY), (crest, 67, CREST BKS/PXD TWN PCK), (crest, 67, CREST TOOTHPASTE GEL), (crest, 67, CREST BKS & PEROX WHITE), (crest, 67, CREST PRO HLT CLN MINT TP), (crest, 67, CREST TUBE REG), (crest, 67, CREST PH ADV HLTH XTR), (crest, 67, CREST PH WHITENING POWR), (crest, 67, CREST 3D BRILLANCE), (crest, 67, CREST PRO-HEALTH RINSE), (crest, 67, CREST FOR KIDS TUBE GEL), (crest, 67, CREST W/DLUX T/B), (crest, 67, CREST PH WHT PWR FRSH TWIN TP), (crest, 67, CREST REG T/P TWN PCK), (crest, 67, CREST PH PRINCESS PASTE), (crest, 67, CREST PH ADV TPRT MNT), (crest, 67, CREST TC WHT CL MNT PST), (crest, 67, CREST T/P TARTER PROT), (crest, 67, CREST PH ADV XT DC MW), (crest, 67, CREST KID SESAME STREET TBRUSH), (crest, 67, CREST P/H INVIG MNT RIN), (crest, 67, CREST CAV PROT REG TP), (crest, 67, CREST PH ADV DP CLN), (crest, 67, CREST COMP WHT+SCOPE TWIN), (crest, 67, CREST REG.TRIAL SIZE), (crest, 67, CREST PROHLTH RINSE), (crest, 67, CREST 3DWHT ARTC FRSH), (crest, 67, CREST COMPL WHITNG HRBL), ...]

[200 rows x 0 columns]

In [ ]:
final_answer_df = pd.read_csv('Q3_Answer.csv')
final_answer_df

,ngram,counts,description
0,choc,207,HER SOILD M/CHOC HEART
1,choc,207,HER M/CHOC MINI KISS
2,choc,207,HUNTS SNK PACK NAT CHOC
3,choc,207,GOOD HUMOR CHOC ECLAIRS
4,choc,207,BAKR DIPPNG CHOC MILK C
...,...,...,...
4816,mx,66,PLNTR NUTRTN DIGSTV MX
4817,mx,66,CRM WHT MX&EAT ORIG PKT
4818,mx,66,MCC PICKLING SPICE MX
4819,mx,66,KNORR SCE MX GARLC HERB
